<a href="https://colab.research.google.com/github/GaboLara998/ProyectoIntegrador/blob/main/ProyectoIntregrador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carga de Datos

In [1]:
# Carga de Datasets
import pandas as pd
import warnings
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.preprocessing import label_binarize

warnings.filterwarnings('ignore')

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [2]:
!pip install -U imbalanced-learn

In [3]:
#Funcion de cargar datasets
def cargar_datos(url):
    datos = pd.read_csv(url, encoding='latin1', skipinitialspace=True)
    return datos

# Función para eliminar columnas no deseadas
def eliminar_columnas(datos, columnas_a_eliminar):
    return datos.drop(columns=columnas_a_eliminar, errors='ignore')

# Función para verificar y manejar valores faltantes
def manejar_valores_faltantes(datos):
    for columna in datos.columns:
        if datos[columna].isnull().any():
            datos[columna].fillna(datos[columna].mode()[0], inplace=True)
    return datos

# Función para codificar variables categóricas
def codificar_variables_categoricas(datos, columnas_categoricas):
    encoder = LabelEncoder()
    for columna in columnas_categoricas:
        datos[columna] = encoder.fit_transform(datos[columna])
    return datos

# Función para normalizar todas las características numéricas de un DataFrame
def normalizar_caracteristicas(datos, columnas_categoricas):
    encoder = LabelEncoder()
    scaler = MinMaxScaler()

    for columna in columnas_categoricas:
        datos[columna] = encoder.fit_transform(datos[columna])

    columnas_numericas = datos.select_dtypes(include=['float64', 'int64']).columns
    datos[columnas_numericas] = scaler.fit_transform(datos[columnas_numericas])

    return datos


#Autoencoders

def entrenar_autoencoder(features, dimensiones_intermedias=[32, 16], epochs=50, batch_size=256):
    """
    Entrena un autoencoder y devuelve el modelo junto con el encoder.

    Args:
    - features: Características de entrada para entrenar el autoencoder.
    - dimensiones_intermedias: Lista de enteros que define las dimensiones de las capas intermedias.
    - epochs: Número de épocas para entrenar el autoencoder.
    - batch_size: Tamaño del lote para el entrenamiento.

    Returns:
    - autoencoder: Modelo de autoencoder entrenado.
    - encoder: Parte del encoder del autoencoder para la reducción de la dimensionalidad.
    """
    input_dim = features.shape[1]
    input_layer = Input(shape=(input_dim,))
    encoder_layer = input_layer
    for dim in dimensiones_intermedias:
        encoder_layer = Dense(dim, activation="relu")(encoder_layer)
    decoder_layer = Dense(dimensiones_intermedias[-2], activation="relu")(encoder_layer)
    for dim in reversed(dimensiones_intermedias[:-1]):
        decoder_layer = Dense(dim, activation="relu")(decoder_layer)
    decoder_layer = Dense(input_dim, activation="sigmoid")(decoder_layer)

    autoencoder = Model(input_layer, decoder_layer)
    autoencoder.compile(optimizer='adam', loss='mean_squared_error')
    autoencoder.fit(features, features, epochs=epochs, batch_size=batch_size, shuffle=True, validation_split=0.2)

    encoder = Model(input_layer, encoder_layer)
    return autoencoder, encoder

def transformar_con_autoencoder(encoder, features):
    """
    Transforma las características utilizando el encoder de un autoencoder entrenado.

    Args:
    - encoder: El encoder de un autoencoder.
    - features: Características a transformar.

    Returns:
    - Características transformadas.
    """
    return encoder.predict(features)

In [5]:
# URLs de los conjuntos de datos
url_training = "https://raw.githubusercontent.com/GaboLara998/ProyectoIntegrador/main/DataSet_UNSW_NB15/UNSW_NB15_training-set.csv?token=GHSAT0AAAAAACMBWAOFBU5CFZ67HIDC2XLOZO7JZMA"
url_testing = "https://raw.githubusercontent.com/GaboLara998/ProyectoIntegrador/main/DataSet_UNSW_NB15/UNSW_NB15_testing-set.csv?token=GHSAT0AAAAAACMBWAOFEFKXDJN42NYN2ZDAZO7JXNA"




# Cargar los datos
datos_training = cargar_datos(url_training)
datos_testing = cargar_datos(url_testing)

# Columnas sin considerar
columnas_a_eliminar = ['id']
# Columnas categóricas
columnas_categoricas = ['proto', 'service', 'state', 'attack_cat']

# Eliminar columnas no deseadas
datos_training = eliminar_columnas(datos_training, columnas_a_eliminar)
datos_testing = eliminar_columnas(datos_testing, columnas_a_eliminar)

# Manejar valores faltantes
datos_training = manejar_valores_faltantes(datos_training)
datos_testing = manejar_valores_faltantes(datos_testing)

# Preprocesar datos
datos_training = normalizar_caracteristicas(datos_training, columnas_categoricas)
datos_testing = normalizar_caracteristicas(datos_testing, columnas_categoricas)



# Separa las características y las etiquetas en el conjunto de entrenamiento
X = datos_training.drop('label', axis=1)
y = datos_training['label']

# Aplicar SMOTE para tratar el desequilibrio de clases en el conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_sm, y_sm = smote.fit_resample(X, y)

# Ahora proceder a entrenar tu autoencoder y otros modelos con X_sm y y_sm
# Entrenamiento y transformación con Autoencoder
autoencoder, encoder = entrenar_autoencoder(X_sm.to_numpy())
features_training_reduced = transformar_con_autoencoder(encoder, X_sm.to_numpy())





Epoch 1/50
746/746 [==============================] - 4s 4ms/step - loss: 0.0232 - val_loss: 0.0056
Epoch 2/50
746/746 [==============================] - 3s 4ms/step - loss: 0.0034 - val_loss: 0.0023
Epoch 3/50
746/746 [==============================] - 4s 5ms/step - loss: 0.0016 - val_loss: 0.0013
Epoch 4/50
746/746 [==============================] - 3s 4ms/step - loss: 0.0011 - val_loss: 0.0010
Epoch 5/50
746/746 [==============================] - 3s 4ms/step - loss: 9.3647e-04 - val_loss: 8.7319e-04
Epoch 6/50
746/746 [==============================] - 3s 4ms/step - loss: 8.0110e-04 - val_loss: 7.7122e-04
Epoch 7/50
746/746 [==============================] - 4s 5ms/step - loss: 7.0126e-04 - val_loss: 6.9989e-04
Epoch 8/50
746/746 [==============================] - 3s 4ms/step - loss: 6.4648e-04 - val_loss: 6.3415e-04
Epoch 9/50
746/746 [==============================] - 2s 3ms/step - loss: 6.0596e-04 - val_loss: 5.9346e-04
Epoch 10/50
746/746 [==============================] - 2s 3m

In [8]:
from sklearn.metrics import make_scorer, mean_squared_error

# Definir la métrica de puntuación personalizada
scorer = make_scorer(mean_squared_error)

# Crear el objeto GridSearchCV con la métrica de puntuación personalizada
grid_search = GridSearchCV(IsolationForest(random_state=42), param_grid, cv=5, scoring=scorer)

# Ajustar el modelo a los datos
grid_search.fit(features_training_reduced)

# Imprimir los mejores parámetros
print(grid_search.best_params_)
#{'contamination': 'auto', 'n_estimators': 50}


{'contamination': 'auto', 'n_estimators': 50}


In [ ]:
print(datos_testing.describe())
print(datos_testing.info())

In [ ]:
print(datos_training.describe())
print(datos_training.info())

# Implementación de Algoritmos

In [7]:
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM

# No considerarar la columna de etiquetas antes de entrenar el autoencoder
features_training = datos_training.drop(columns=['label'], errors='ignore')
features_testing = datos_testing.drop(columns=['label'], errors='ignore')

# Entrenamiento y transformación con Autoencoder
# Asegúrate de convertir DataFrame a numpy array si tu función esperar explícitamente numpy arrays
autoencoder, encoder = entrenar_autoencoder(features_training.to_numpy())
features_training_reduced = transformar_con_autoencoder(encoder, features_training.to_numpy())
features_testing_reduced = transformar_con_autoencoder(encoder, features_testing.to_numpy())

# Entrenamiento de Isolation Forest y One-Class SVM con características reducidas
isolation_forest = IsolationForest(n_estimators=100, contamination='auto', random_state=42)
isolation_forest.fit(features_training_reduced)

one_class_svm = OneClassSVM(kernel='rbf', gamma='auto')
one_class_svm.fit(features_training_reduced)

Epoch 1/50
548/548 [==============================] - 3s 4ms/step - loss: 0.0271 - val_loss: 0.0042
Epoch 2/50
548/548 [==============================] - 2s 3ms/step - loss: 0.0029 - val_loss: 0.0026
Epoch 3/50
548/548 [==============================] - 2s 3ms/step - loss: 0.0020 - val_loss: 0.0017
Epoch 4/50
548/548 [==============================] - 3s 6ms/step - loss: 0.0015 - val_loss: 0.0011
Epoch 5/50
548/548 [==============================] - 3s 5ms/step - loss: 0.0012 - val_loss: 9.1751e-04
Epoch 6/50
548/548 [==============================] - 2s 3ms/step - loss: 9.5680e-04 - val_loss: 7.5332e-04
Epoch 7/50
548/548 [==============================] - 2s 3ms/step - loss: 8.4210e-04 - val_loss: 6.5219e-04
Epoch 8/50
548/548 [==============================] - 2s 3ms/step - loss: 7.6232e-04 - val_loss: 5.9293e-04
Epoch 9/50
548/548 [==============================] - 2s 3ms/step - loss: 6.8803e-04 - val_loss: 5.0274e-04
Epoch 10/50
548/548 [==============================] - 2s 3ms/st

OneClassSVM(gamma='auto')

# Preprocesamiento de Datos

In [9]:
# Entrenamiento de Isolation Forest y One-Class SVM con características reducidas
isolation_forest = IsolationForest(n_estimators=50, contamination='auto', random_state=42)
isolation_forest.fit(features_training_reduced)

one_class_svm = OneClassSVM(kernel='rbf', gamma='auto')
one_class_svm.fit(features_training_reduced)

# Transformación de los datos de prueba
features_testing_reduced = transformar_con_autoencoder(encoder, datos_testing.drop('label', axis=1).to_numpy())

# Predicciones para los modelos
predictions_isolation_forest = isolation_forest.predict(features_testing_reduced)
predictions_one_class_svm = one_class_svm.predict(features_testing_reduced)

# Convertir las predicciones de -1, 1 a 0, 1
predictions_isolation_forest_binarized = (predictions_isolation_forest < 0).astype(int)
predictions_one_class_svm_binarized = (predictions_one_class_svm < 0).astype(int)

# Etiquetas de prueba reales
labels_testing_binarized = label_binarize(datos_testing['label'], classes=[0, 1]).ravel()

# Cálculo de métricas
precision_if = precision_score(labels_testing_binarized, predictions_isolation_forest_binarized)
recall_if = recall_score(labels_testing_binarized, predictions_isolation_forest_binarized)
f1_score_if = f1_score(labels_testing_binarized, predictions_isolation_forest_binarized)

precision_svm = precision_score(labels_testing_binarized, predictions_one_class_svm_binarized)
recall_svm = recall_score(labels_testing_binarized, predictions_one_class_svm_binarized)
f1_score_svm = f1_score(labels_testing_binarized, predictions_one_class_svm_binarized)

print(f"Isolation Forest - Precisión: {precision_if}, Recall: {recall_if}, F1-Score: {f1_score_if}")
print(f"One-Class SVM - Precisión: {precision_svm}, Recall: {recall_svm}, F1-Score: {f1_score_svm}")


2573/2573 [==============================] - 4s 1ms/step
Isolation Forest - Precisión: 0.4643794343473613, Recall: 0.10540015882820084, F1-Score: 0.17180561298790026
One-Class SVM - Precisión: 0.6602087592145203, Recall: 0.940417365216624, F1-Score: 0.7757861406318243


In [ ]:
# Revisar balanceo de datos
# Random Forest y normalizacion de datos
# seleccion de mejores caracteristicas
# extraer las caracteristicas
# Comparar efectividad


#Mejorar resultados usar busqeuda de cuadricula o busqueda aleatorioa.
# seleccionar caracteristicas como el PCA

#Balanceo de Clases


In [ ]:
#Isolation Forest
#from sklearn.ensemble import IsolationForest

# Instancia del modelo Isolation Forest
#isolation_forest = IsolationForest(n_estimators=100, contamination='auto', random_state=42)

# Entrenamiento del modelo con los datos de entrenamiento (sin incluir la etiqueta)
# Asumiendo que 'label' es la columna de etiquetas y ha sido removida de features_training_normalized
#isolation_forest.fit(datos_training)


#OneClassSVM
#from sklearn.svm import OneClassSVM

# Instancia del modelo One-Class SVM
#one_class_svm = OneClassSVM(kernel='rbf', gamma='auto')

# Entrenamiento del modelo con los datos de entrenamiento
#one_class_svm.fit(datos_training)